In [1]:
import datetime
import os
import pandas as pd
import numpy 
from haversine import haversine
import json
import numpy as np
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

In [2]:
sotf_excel=pd.ExcelFile("./BL SOTF 10.12 Zips .xlsx")
sotf_excel.sheet_names

['BL_10.12 Zips_51 Stores', 'BL_10.26 Zips_3 Stores']

In [3]:
df_sotf_1012=sotf_excel.parse('BL_10.12 Zips_51 Stores',dtype=str)
df_sotf_1026=sotf_excel.parse('BL_10.26 Zips_3 Stores',dtype=str)


In [4]:
TA_excel=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_remove_8_and_22_stores_JL_2019-09-11.xlsx")
TA_excel.sheet_names

['unique_zips_full_footprint', 'view_by_TA', 'view_by_store', 'stores_removed']

In [5]:
store_ta_zips=TA_excel.parse("view_by_store",dtype=str)
store_ta_zips.shape

(1378, 17)

In [6]:
store_list_1012=df_sotf_1012['location_id'].unique().tolist()
store_list_1026=df_sotf_1026['location_id'].unique().tolist()

all_sotf=store_list_1012+store_list_1026

In [7]:
df_existing=store_ta_zips[store_ta_zips['location_id'].isin(all_sotf)]


In [8]:
store_list_new=[x for x in all_sotf if x not in df_existing['location_id'].tolist()]
print(store_list_new)

['4705', '4706', '4702', '4678', '5407', '5402', '5398', '5408']


In [9]:
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190901-135942-508.txt",dtype=str,sep="|")
store_list['latitude_meas']=store_list['latitude_meas'].astype(float)
store_list['longitude_meas']=store_list['longitude_meas'].astype(float)
store_list['zip_cd']=store_list['zip_cd'].apply(lambda x: x.strip().split("-")[0].zfill(5))
store_list=store_list.rename(columns={"zip_cd":"store_zip"})
store_list[store_list['location_id'].isin(store_list_new)]
# Empty df=all not opened stores

,location_id,location_desc,open_dt,close_dt,address_line_1,address_line_2,city_nm,state_nm,store_zip,longitude_meas,latitude_meas,sales_square_feet,furniture_sales_square_feet


In [10]:
# Google the location for new stores
dict_new_store_address={}

# 4705 	370 AUTO CENTER CIRCLE	NORTHRIDGE MALL	SALINAS	CA	93907
dict_new_store_address.update({"4705":{"GoogleAddress":"370 Auto Center Cir, Salinas, CA 93906","GoogleCoor":[36.7148622,-121.6607234]}})

# 4706 	3170 CHILI AVE UNIT # T5	CHILI PLAZA	ROCHESTER	NY	14624-5410
dict_new_store_address.update({"4706":{"GoogleAddress":"3170 Chili Ave T5, Rochester, NY 14624","GoogleCoor":[43.1085609,-77.7467213]}})

# 4702 	6375 MONTANA AVENUE	KMART SHOPPING CENTER	EL PASO	TX	79925-2044
dict_new_store_address.update({"4702":{"GoogleAddress":"6375 Montana Ave, El Paso, TX 79925","GoogleCoor":[31.7898616,-106.404997]}})

# 4678 	3501 EAST LINCOLNWAY	CHEYENNE PLAZA	CHEYENNE	WY	82001-6302
dict_new_store_address.update({"4678":{"GoogleAddress":"3501 E Lincolnway, Cheyenne, WY 82001","GoogleCoor":[41.1370675,-104.7674035]}})

# 5407 	14154 EAST WADE HAMPTON BLVD	COLLINS CENTER	GREER	SC	29651-1554
dict_new_store_address.update({"5407":{"GoogleAddress":"14154 E Wade Hampton Blvd, Greer, SC 29651","GoogleCoor":[34.948923,-82.2118793]}})

# 5402 	14603 TELEGRAPH ROAD	TOYS R US CENTER	WOODBRIDGE	VA	22192-4601
dict_new_store_address.update({"5402":{"GoogleAddress":"14603 Telegraph Rd, Woodbridge, VA 22194","GoogleCoor":[38.6373899,-77.2914551]}})

# 5398 	275 ROUTE 18	MIRACLE MALL	EAST BRUNSWICK	NJ	08816-1904
dict_new_store_address.update({"5398":{"GoogleAddress":"275 NJ-18, East Brunswick, NJ 08816","GoogleCoor":[40.4592229,-74.4049486]}})

# 5408 	3420 SOUTHWEST DURHAM DRIVE	PATTERSON PLACE	DURHAM	NC	27707-3351
dict_new_store_address.update({"5408":{"GoogleAddress":"3420 SW Durham Dr, Durham, NC 27707","GoogleCoor":[35.9516434,-78.9887268]}})


In [11]:
dict_new_store_address

{'4678': {'GoogleAddress': '3501 E Lincolnway, Cheyenne, WY 82001',
  'GoogleCoor': [41.1370675, -104.7674035]},
 '4702': {'GoogleAddress': '6375 Montana Ave, El Paso, TX 79925',
  'GoogleCoor': [31.7898616, -106.404997]},
 '4705': {'GoogleAddress': '370 Auto Center Cir, Salinas, CA 93906',
  'GoogleCoor': [36.7148622, -121.6607234]},
 '4706': {'GoogleAddress': '3170 Chili Ave T5, Rochester, NY 14624',
  'GoogleCoor': [43.1085609, -77.7467213]},
 '5398': {'GoogleAddress': '275 NJ-18, East Brunswick, NJ 08816',
  'GoogleCoor': [40.4592229, -74.4049486]},
 '5402': {'GoogleAddress': '14603 Telegraph Rd, Woodbridge, VA 22194',
  'GoogleCoor': [38.6373899, -77.2914551]},
 '5407': {'GoogleAddress': '14154 E Wade Hampton Blvd, Greer, SC 29651',
  'GoogleCoor': [34.948923, -82.2118793]},
 '5408': {'GoogleAddress': '3420 SW Durham Dr, Durham, NC 27707',
  'GoogleCoor': [35.9516434, -78.9887268]}}

In [12]:
df_output_existing=store_list[['location_id','state_nm','store_zip','latitude_meas','longitude_meas']]
df_output_existing=df_output_existing[df_output_existing['location_id'].isin(df_existing['location_id'].unique().tolist())]
df_zip_cd=pd.DataFrame()
df_existing=df_existing.reset_index()
del df_existing['index']
for ind,row in df_existing.iterrows():
    store_num=row['location_id']
    zip_list_P=eval(row['trans_P_zips_70_within_TA'])
    zip_list_S=eval(row['trans_S_zips_70_within_TA'])
    zip_list_10=eval(row['zips_in_10'])
    
    df_P=pd.DataFrame({"zip":zip_list_P},index=["P"]*len(zip_list_P))
    df_S=pd.DataFrame({"zip":zip_list_S},index=["S"]*len(zip_list_S))
    df_10=pd.DataFrame({"zip":zip_list_10},index=["zip_in_10"]*len(zip_list_10))
    
    df=pd.concat([df_P,df_S,df_10]).reset_index().rename(columns={"index":"trans_flag"})
    df['location_id']=store_num
    df_zip_cd=df_zip_cd.append(df)
df_output_existing=pd.merge(df_output_existing,df_zip_cd,on="location_id",how="left")

In [13]:
df_output_existing[(df_output_existing['latitude_meas']==0) | (df_output_existing['longitude_meas']==0)]['location_id'].unique().tolist()

[]

In [15]:
df_output_existing['Distance (mile)']=np.nan
df_output_existing=df_output_existing.reset_index()
del df_output_existing['index']

for ind, row in df_output_existing.iterrows():
    dist=haversine((row['latitude_meas'],row['longitude_meas']),zip_centers[row['zip']],miles=True)
    df_output_existing.loc[ind,"Distance (mile)"]=dist
    

In [20]:
df_output_new=pd.DataFrame()
for location_id in store_list_new:
    state_nm=dict_new_store_address[location_id]['GoogleAddress'].split(", ")[-1][:2]
    store_zip=dict_new_store_address[location_id]['GoogleAddress'].split(", ")[-1].split(" ")[1]
    latitude_meas=dict_new_store_address[location_id]['GoogleCoor'][0]
    longitude_meas=dict_new_store_address[location_id]['GoogleCoor'][1]
    df=pd.DataFrame()
    for zip_cd,v in zip_centers.items():
        dist=haversine((latitude_meas,longitude_meas),v,miles=True)
        if dist<=10:
            df=df.append(pd.DataFrame({"zip":zip_cd,"Distance (mile)":dist},index=["zip_in_10"]))
    df=df.reset_index().rename(columns={"index":"trans_flag"})
    df['location_id']=location_id
    df['state_nm']=state_nm
    df['store_zip']=store_zip
    df['latitude_meas']=latitude_meas
    df['longitude_meas']=longitude_meas
    df_output_new=df_output_new.append(df)
    
            

In [28]:
df_output_all=df_output_existing.append(df_output_new)
df_output_all['G.O.']=np.nan
col_list=df_sotf_1012.columns.tolist()

col_list_new=[]
for col in col_list:
    if col=="revenue_flag":
        new_col="trans_flag"
    else:
        new_col=col
    col_list_new=col_list_new+[new_col]
df_output_all=df_output_all[col_list_new]

In [30]:
df_output_all['G.O.']=np.where(df_output_all['location_id'].isin(store_list_1012),"1012",
                              np.where(df_output_all['location_id'].isin(store_list_1026),"1026",np.nan)
                              )
df_output_1012=df_output_all[df_output_all['G.O.']=="1012"]
df_output_1026=df_output_all[df_output_all['G.O.']=="1026"]


In [37]:
df_unique_zips_1012=df_output_1012[['trans_flag','zip']].sort_values(["zip","trans_flag"]).drop_duplicates("zip")
df_unique_zips_1026=df_output_1026[['trans_flag','zip']].sort_values(["zip","trans_flag"]).drop_duplicates("zip")

In [38]:
writer=pd.ExcelWriter("./BL_SOTF 10.12 Zips_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_unique_zips_1012.to_excel(writer,'unique_zip_1012',index=False)
df_unique_zips_1026.to_excel(writer,'unique_zip_1026',index=False)

df_output_1012.to_excel(writer,'BL_10.12 Zips_51 Stores',index=False)
df_output_1026.to_excel(writer,'BL_10.26 Zips_3 Stores',index=False)
writer.save()